# 1.   Определение стоимости автомобилей

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 
<br><br>Заказчику важны:
    
- качество предсказания;
- скорость предсказания;
- время обучения.</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Данные представлены в виде датасета со многими параметрами и ключевым параметром цены. В данных много пропусков, также присутствуют ненужные для обучения параметры
<br><br>Задачей проекта является очистка данных, подготовка к обучению моделей, а также обучение и сравнение полученных результатов
<br><br>Целью проекта является определение лучшей модели машинного обучения для данной задачи
<br><br>План работы:
    
- Подготовка данных, анализ и очистка
- Обучение моделей, простых линейных и бустов
- Анализ моделей для поиска подходящей</div>

## 1.1.   Подготовка данных

In [1]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor

In [2]:
data = pd.read_csv('/datasets/autos.csv')

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

### 1.1.1.   Удаление колонок

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Похоже, что в колонках DateCrawled, DateCreated и LastSeen надобности нет, и их можно удалить</div>

In [5]:
data = data.drop(['DateCrawled', 'DateCreated', 'LastSeen', 
                  'PostalCode', 'RegistrationMonth'], axis=1)

### 1.1.2.   Обработка ошибок

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Сделаем краткий обзор колонок с пропущенными данными</div>

In [6]:
print('Количество ошибок в колонке:\n')
for column in data.columns:
    nans = len(data[data[column].isna()][column])
    if nans:
        print(f'{column}: {nans}, ({round(nans/len(data), 2)*100}%)')

Количество ошибок в колонке:

VehicleType: 37490, (11.0%)
Gearbox: 19833, (6.0%)
Model: 19705, (6.0%)
FuelType: 32895, (9.0%)
NotRepaired: 71154, (20.0%)


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>Vehicle Type
<br><br></div>

In [7]:
data['VehicleType'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Здесь все достаточно просто – пропуски можно заменить на тип 'unknown', так как данные действительно неизвестны</div>

In [8]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')

In [9]:
data[data['VehicleType'].isna()]

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>Gearbox
<br><br></div>

In [10]:
data['Gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Здесь, скорее всего, придется поступить тем же образом – заменить данные на 'unknown', но коробка, конечно, очень важна для оценки стоимости автомобиля</div>

In [11]:
data['Gearbox'] = data['Gearbox'].fillna('unknown')

In [12]:
data[data['Gearbox'].isna()]

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>Model
<br><br></div>

In [13]:
data['Model'].nunique()

250

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
250 уникальных показателей, и, скорее всего, такая же методика обработки</div>

In [14]:
data['Model'] = data['Model'].fillna('unknown')

In [15]:
data[data['Model'].isna()]

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>FuelType
<br><br></div>

In [16]:
data['FuelType'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Отработанными движениями заменяем неизвестные</div>

In [17]:
data['FuelType'] = data['FuelType'].fillna('unknown')

In [18]:
data[data['FuelType'].isna()]

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>NotRepaired
<br><br></div>

In [19]:
data['NotRepaired'].unique()

array([nan, 'yes', 'no'], dtype=object)

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Можно предположить, что если не указана информация о восстановлении, значит, это не восстановленный автомобиль</div>

In [20]:
data['NotRepaired'] = data['NotRepaired'].fillna('no')

In [21]:
data[data['NotRepaired'].isna()]

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Перепроверка</div>

In [22]:
print('Количество ошибок в колонке:\n')
for column in data.columns:
    nans = len(data[data[column].isna()][column])
    if nans:
        print(f'{column}: {nans}, ({round(nans/len(data), 2)*100}%)')

Количество ошибок в колонке:



<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Ошибок нет</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>Price
<br><br></div>

In [23]:
print(data.Price.sort_values().unique()[:200])

[  0   1   2   3   4   5   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  24  25  26  27  29  30  32  33  35  38  39  40  45  47  49
  50  55  58  59  60  65  66  69  70  74  75  77  79  80  85  88  89  90
  95  98  99 100 101 105 106 108 110 111 112 115 117 119 120 121 122 123
 125 126 127 128 129 130 132 133 135 139 140 142 145 149 150 155 156 157
 158 159 160 162 165 166 169 170 173 175 177 179 180 181 185 188 189 190
 192 193 195 196 198 199 200 202 205 209 210 211 215 217 219 220 222 224
 225 229 230 233 235 236 238 240 243 245 248 249 250 251 252 253 255 259
 260 261 263 265 266 269 270 272 273 274 275 277 278 279 280 281 284 285
 288 289 290 293 295 298 299 300 301 305 308 309 310 315 316 319 320 321
 323 325 329 330 331 332 333 335 338 339 340 341 344 345 348 349 350 355
 356 359]


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Много цен в евро, которые слабо соответствуют ценам даже самого простого автомобиля. Слабо понятно, откуда они взялись, но можно попробовать сделать срез данных начиная с 300 евро</div>

In [24]:
data = data[data.Price >= 300]

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>Power
<br><br></div>

In [25]:
print(data.Power.sort_values().unique()[:40])
print()
print(data.Power.sort_values(ascending=False).unique()[:40])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]

[20000 19312 19211 19208 17932 17700 17410 17019 17011 16312 16311 16051
 16011 15033 15020 15017 15001 14009 13636 13616 12512 12510 12012 11635
 11530 11509 11025 11011 10912 10910 10710 10522 10520 10311 10218 10110
 10000  9013  9012  9011]


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Очень странные показатели л.с. Можно оставить от 80 (малолитражки) до 400 (спорткары и премиум), большинство из остальных (даже если мощность реальная) все равно ценой слабо коррелируют со своими показателями</div>

In [26]:
data = data[(data.Power >= 80) & (data.Power <= 400)]

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
    <br>RegistrationYear
<br><br></div>

In [27]:
data.RegistrationYear.sort_values().unique()

array([1000, 1500, 1800, 1910, 1937, 1947, 1948, 1949, 1950, 1951, 1952,
       1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,
       1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2800, 2900, 4500, 5000, 6000, 8200, 9999])

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Первые три значения все равно удалять – первый автомобиль был произведен в 1886 г. 
<br><br>Остальные можно срезать, например, с 1990 – допускаем, что 30-летние автомобили вполне на ходу. Остальное, скорее всего, будет считаться раритетом для коллекции, и ценой также слабо зависеть от показателей</div>

In [28]:
data = data[(data.RegistrationYear >= 1990) & (data.RegistrationYear <= 2022)]

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Reset индексов для отсутствия ошибок (объяснено в комментарии чуть ниже, на моменте OE)</div>

In [29]:
data = data.reset_index(drop=True)

### 1.1.3.   Типы данных

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220375 entries, 0 to 220374
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             220375 non-null  int64 
 1   VehicleType       220375 non-null  object
 2   RegistrationYear  220375 non-null  int64 
 3   Gearbox           220375 non-null  object
 4   Power             220375 non-null  int64 
 5   Model             220375 non-null  object
 6   Kilometer         220375 non-null  int64 
 7   FuelType          220375 non-null  object
 8   Brand             220375 non-null  object
 9   NotRepaired       220375 non-null  object
 10  NumberOfPictures  220375 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 18.5+ MB


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Посмотрим на количества уникальных значений в объектах (будем брать значение на 1 меньше, так как при OneHot можно выбросить первую колонку)</div>

In [31]:
print('Количество уникальных значений:\n')
for column in data.columns:
    if data[column].dtype == 'object':
        print(f'{column}: {data[column].nunique()-1}')

Количество уникальных значений:

VehicleType: 8
Gearbox: 2
Model: 241
FuelType: 7
Brand: 38
NotRepaired: 1


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Столбцы Gearbox и NotRepaired точно подходят под OneHot, так как добавится лишь 3 новых столбца (при условии, что GearBox не будет пополняться новыми значениями), а остальные выгоднее декодировать в числа внутри столбца, так как ни 8, ни 250 новых колонок нет необходимости создавать</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>OHE
<br><br></div>

In [32]:
# перезаписываем data
data = (pd
        # добавляем новые столбцы OHE
        .concat([data, pd.get_dummies(data[['Gearbox', 'NotRepaired']], 
                                      # выбрасываем первый
                                      drop_first=True)], axis=1)
        # удаляем старые
        .drop(['Gearbox', 'NotRepaired'], axis=1))

In [33]:
data.head()

,Price,VehicleType,RegistrationYear,Power,Model,Kilometer,FuelType,Brand,NumberOfPictures,Gearbox_manual,Gearbox_unknown,NotRepaired_yes
0,18300,coupe,2011,190,unknown,125000,gasoline,audi,0,1,0,1
1,9800,suv,2004,163,grand,125000,gasoline,jeep,0,0,0,0
2,650,sedan,1995,102,3er,150000,petrol,bmw,0,1,0,1
3,2200,convertible,2004,109,2_reihe,150000,petrol,peugeot,0,1,0,0
4,14500,bus,2014,125,c_max,30000,petrol,ford,0,1,0,0


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
<br>Ordinal
<br><br></div>

In [34]:
encoder = OrdinalEncoder()

In [35]:
encoder.fit(data[['VehicleType', 'Model', 'FuelType', 'Brand']])

OrdinalEncoder()

In [36]:
ordinal_data = pd.DataFrame(encoder.transform(data[['VehicleType', 'Model', 'FuelType', 'Brand']]), 
                            columns=['VehicleType', 'Model', 'FuelType', 'Brand'])

In [37]:
ordinal_data.head()

,VehicleType,Model,FuelType,Brand
0,2.0,219.0,2.0,1.0
1,6.0,114.0,2.0,14.0
2,4.0,11.0,6.0,2.0
3,1.0,8.0,6.0,25.0
4,0.0,59.0,6.0,10.0


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Похоже на то, что нам необходимо</div>

In [38]:
data[['VehicleType', 'Model', 'FuelType', 'Brand']] = ordinal_data

In [39]:
data.head()

,Price,VehicleType,RegistrationYear,Power,Model,Kilometer,FuelType,Brand,NumberOfPictures,Gearbox_manual,Gearbox_unknown,NotRepaired_yes
0,18300,2.0,2011,190,219.0,125000,2.0,1.0,0,1,0,1
1,9800,6.0,2004,163,114.0,125000,2.0,14.0,0,0,0,0
2,650,4.0,1995,102,11.0,150000,6.0,2.0,0,1,0,1
3,2200,1.0,2004,109,8.0,150000,6.0,25.0,0,1,0,0
4,14500,0.0,2014,125,59.0,30000,6.0,10.0,0,1,0,0


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220375 entries, 0 to 220374
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             220375 non-null  int64  
 1   VehicleType       220375 non-null  float64
 2   RegistrationYear  220375 non-null  int64  
 3   Power             220375 non-null  int64  
 4   Model             220375 non-null  float64
 5   Kilometer         220375 non-null  int64  
 6   FuelType          220375 non-null  float64
 7   Brand             220375 non-null  float64
 8   NumberOfPictures  220375 non-null  int64  
 9   Gearbox_manual    220375 non-null  uint8  
 10  Gearbox_unknown   220375 non-null  uint8  
 11  NotRepaired_yes   220375 non-null  uint8  
dtypes: float64(4), int64(5), uint8(3)
memory usage: 15.8 MB


## 1.2.   Обучение моделей

### 1.2.1.   Разделение данных

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Разделим данные на тренировочную и валидационную части</div>

In [41]:
Features = data.drop('Price', axis=1)
Target = data['Price']

In [42]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(Features, Target, 
                                                                            test_size=0.4, 
                                                                            random_state=12345)

In [43]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, 
                                                                            test_size=0.5, 
                                                                            random_state=12345)

In [44]:
print(features_train.shape, target_train.shape)
print(features_valid.shape, target_valid.shape)
print(features_test.shape, target_test.shape)

(132225, 11) (132225,)
(44075, 11) (44075,)
(44075, 11) (44075,)


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Размеры совпадают, можно обучать</div>

### 1.2.2.   LinearRegression

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Обучим сначала обычную линейную регрессию</div>

In [45]:
linear_model = LinearRegression()

In [46]:
%%time
linear_model.fit(features_train, target_train)

CPU times: user 42.8 ms, sys: 27.7 ms, total: 70.5 ms
Wall time: 52 ms


LinearRegression()

In [47]:
predicted = linear_model.predict(features_valid)

In [48]:
mse(predicted, target_valid)**0.5

2921.638862562086

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Довольно большой разброс при максимальной цене в 20000</div>

### 1.2.4.   CatBoostRegressor

In [54]:
best_mse = None
best_depth = None
best_estimators = None
i = 0

for max_depth in range(14, 17):
    for n_estimators in range(1040, 1061, 10):
        catboost_model = CatBoostRegressor(loss_function='RMSE', 
                                           max_depth=max_depth, 
                                           n_estimators=n_estimators)
        catboost_model.fit(features_train, target_train, verbose=0)
        predicted = catboost_model.predict(features_valid)
        catboost_mse = mse(predicted, target_valid)**0.5
        if not best_mse:
            best_mse = catboost_mse
        elif catboost_mse < best_mse:
            best_mse = catboost_mse
            best_depth = max_depth
            best_estimators = n_estimators
        i += 1
        print(i, end=' ... ')
        
print(f'\nbest mse: {best_mse}, best depth: {best_depth}, best estimators: {best_estimators}')

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Лучшие результаты:
<br><br>- max_depth: 16 (максимальное значение)
<br><br>- n_estimators: 1050 (чуть больше, чем стандартный)</div>

### 1.2.5.   LGBMRegressor

In [59]:
lightgbm_model = LGBMRegressor()

In [60]:
%%time
lightgbm_model.fit(features_train, target_train)

CPU times: user 7min 1s, sys: 3.29 s, total: 7min 4s
Wall time: 7min 6s


LGBMRegressor()

In [61]:
predicted = lightgbm_model.predict(features_valid)
light_mse = mse(predicted, target_valid)

In [62]:
light_mse**0.5

1818.5899242247274

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
RMSE повыше, но времени затрачено 7 минут</div>

## 1.3.   Анализ моделей

### 1.3.1.   Тренировка на тестовой выборке

In [63]:
test_predicted = lightgbm_model.predict(features_test)
test_light_mse = mse(test_predicted, target_test)

In [64]:
test_light_mse**0.5

1783.2263021033582

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
На тестовой выборке уважаемые показатели, и даже RMSE чуть ниже </div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
LinearRegression, конечно имеет самую высокую скорость работы, но получившийся MSE никак не соответствует ограничениям</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
CatBoost показывает хороший MSE, но обучается ~39 секунд, что~ достаточно долго (учитывая, конечно, объемы данных)</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
LightGBM обучается намного быстрее, однако, почти на 100 превышает RMSE CatBoost</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Фактически, сравнение можно проводить между двумя моделями бустинга, и при сопоставимом в масштабе времени, LightGBM выглядит более пригодным для данной задачи</div>